In [1]:
import numpy as np
import pandas as pd
import tensorflow_datasets as tfds
import tensorflow as tf

In [2]:
raw_train_set, raw_valid_set, raw_test_set = tfds.load(
                                            name="imdb_reviews",
                                            split=["train[:90%]", "train[90%:]", "test"],
                                            as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteWWHV78/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteWWHV78/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteWWHV78/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [3]:
train_set = raw_train_set.shuffle(5000, seed=42).batch(32).prefetch(1)
test_set = raw_test_set.batch(32).prefetch(1)
valid_set = raw_valid_set.batch(32).prefetch(1)

In [4]:
#Lets check some reviews and label.
for text, label in raw_train_set.take(4):
  print(text.numpy().decode("utf-8")[:200])
  print(f"label: {label.numpy()}")

This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting 
label: 0
I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However 
label: 0
Mann photographs the Alberta Rocky Mountains in a superb fashion, and Jimmy Stewart and Walter Brennan give enjoyable performances as they always seem to do. <br /><br />But come on Hollywood - a Moun
label: 0
This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful perf
label: 1


In [5]:
#building vocabulary
vocab_size=1000
text_vec_layer = tf.keras.layers.TextVectorization(max_tokens=vocab_size, standardize='lower_and_strip_punctuation')
text_vec_layer.adapt(train_set.map(lambda text, label: text))

### Model Agnostics

In [6]:
from keras.api._v2.keras import activations
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding

embed_size=128
model = Sequential()
model.add(text_vec_layer)
model.add(Embedding(vocab_size, embed_size, mask_zero=True))
model.add(LSTM(128))
model.add(Dense(1, activation="sigmoid"))

#Compile.
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [11]:
r = model.fit(train_set, validation_data= valid_set, epochs=2)

Epoch 1/2
704/704 [==============================] - 25s 35ms/step - loss: 0.2270 - accuracy: 0.9080 - val_loss: 0.3328 - val_accuracy: 0.8732
Epoch 2/2
704/704 [==============================] - 25s 35ms/step - loss: 0.2095 - accuracy: 0.9175 - val_loss: 0.3462 - val_accuracy: 0.8588


In [12]:
#Testing
model.evaluate(test_set)

782/782 [==============================] - 13s 16ms/step - loss: 0.3408 - accuracy: 0.8614


[0.3408046066761017, 0.8614400029182434]

- A little overfitting we can see.
- best num of epochs to train 8.